## Récupération de pages  aléatoires de Wikipédia dans différentes langues

### Import des bibliothèques nécessaires

In [1]:
import requests
import os
import hashlib
import random

Les quatre bibliothèques importées sont `requests`, `os`, `hashlib` et `random`. Elles fournissent des fonctionnalités pour effectuer des requêtes HTTP, manipuler les fichiers et les dossiers, générer des hachages et effectuer des opérations aléatoires.

### Fonction pour récupérer le résumé d'une page Wikipédia aléatoire dans une langue donnée

In [2]:
def recuperer_page_aleatoire(langue):
    url = f"https://{langue}.wikipedia.org/api/rest_v1/page/random/summary"
    try:
        # Envoi de la requête HTTP GET pour obtenir les données de la page aléatoire
        reponse = requests.get(url)
        reponse.raise_for_status()  # Vérifie si la requête a échoué
        donnees = reponse.json() # Conversion de la réponse en format JSON
        if 'extract' in donnees and 'title' in donnees:
            return donnees['title'], donnees['extract'] ## Retourne le titre et le contenu de la page
        else:
            print(f"Aucun extrait ou titre trouvé pour une page aléatoire en {langue}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de la page aléatoire pour {langue}: {e}")
        return None, None

Fonction `recuperer_page_aleatoire(langue)`:

Cette fonction prend en entrée le code de langue (par exemple, 'fr' pour le français) et récupère le résumé d'une page Wikipédia aléatoire dans cette langue.

Elle utilise l'API de Wikipédia pour obtenir les données de la page aléatoire, extrait le titre et le contenu de la réponse JSON, puis les retourne.

In [3]:
# Fonction pour écrire le contenu d'une page dans un fichier texte
def ecrire_fichier_texte(dossier_data, langue, titre, paragraphe):
    # Génération d'un identifiant unique basé sur le titre de la page
    id_unique = generer_id_unique(titre)
    nom_fichier = f"{id_unique}_{langue}.txt" # Nom du fichier avec l'identifiant et la langue
    chemin_fichier = os.path.join(dossier_data, nom_fichier) # Chemin complet du fichier
    try:
        with open(chemin_fichier, 'w', encoding='utf-8') as fichier:
            premier_paragraphe = paragraphe.split('\n')[0]  # Extraction du premier paragraphe
            fichier.write(premier_paragraphe.strip() + '\n')  # Écriture du premier paragraphe dans le fichier
    except IOError as e:
        print(f"Erreur lors de l'écriture du fichier {chemin_fichier}: {e}")

Fonction `ecrire_fichier_texte(dossier_data, langue, titre, paragraphe)`:

Cette fonction prend en entrée le chemin du dossier de données, la langue de la page, le titre de la page et le contenu de la page.

Elle génère un identifiant unique basé sur le titre de la page, puis écrit le premier paragraphe du contenu dans un fichier texte. La raison pour laquelle j'ai décidé de créer un identifiant unique pour chaque page Wiki récupéréé, c'est que certains titres sont trop longs et leur nom de fichier est donc trop long.

Le fichier est enregistré dans le dossier spécifié.

In [4]:
# Fonction pour générer un identifiant unique basé sur le titre de la page
def generer_id_unique(titre):
    return hashlib.sha1(titre.encode()).hexdigest()[:8]

Fonction `generer_id_unique(titre)`:

Cette fonction prend en entrée le titre de la page et génère un identifiant unique en utilisant la fonction de hachage SHA-1.

L'identifiant unique est tronqué pour ne conserver que les 8 premiers caractères.

In [5]:
def main():
    langues = ['ar', 'de', 'en', 'fr', 'es', 'it', 'ja', 'pt', 'ru', 'ko', 'tr', 'vi', 'zh']
    nb_pages_par_langue = 20
    
    # Liste pour stocker les résultats
    resultats = []
    
    # Création du dossier pour stocker les résultats
    for langue in langues:
        dossier_data = f"./results/"
        os.makedirs(dossier_data, exist_ok=True)
        
        for index in range(1, nb_pages_par_langue + 1):
            titre, contenu = recuperer_page_aleatoire(langue)
            
            if titre and contenu:
                # Ajoutez le titre, le contenu et la langue à la liste des résultats
                resultats.append((titre, contenu, langue))
    
    # Mélangez l'ordre des résultats
    random.shuffle(resultats)
    
    # Parcourez les résultats mélangés et écrivez chaque fichier
    for index, (titre, contenu, langue) in enumerate(resultats, start=1):
        print(f"Contenu de l'article {index} ({langue}):")
        print(contenu)
        ecrire_fichier_texte(dossier_data, langue, titre, contenu)
        print(f"Page enregistrée : {titre}_{langue}.txt")

Fonction `main()`:

Cette fonction est le point d'entrée principal du script.

Elle définit la liste des langues à traiter et le nombre de pages à extraire par langue.

Pour chaque langue, elle crée un dossier pour stocker les résultats, puis extrait les résumés de pages aléatoires.

Les résultats sont stockés dans une liste, mélangés aléatoirement, et pour chaque résultat, le contenu est écrit dans un fichier texte à l'aide des fonctions précédentes.

In [6]:
# Exécution de la fonction principale si le script est exécuté directement
if __name__ == "__main__":
    main()

Contenu de l'article 1 (ko):
오시라카와역(일본어: 大白川駅)은 일본 니가타현 우오누마시에 위치한 동일본 여객철도 다다미 선의 철도역이다. 단선 승강장 1면 1선의 구조를 갖춘 지상역이다.
Page enregistrée : 오시라카와역_ko.txt
Contenu de l'article 2 (fr):
Le xian de Shawan est un district administratif de la région autonome du Xinjiang en Chine. Il est placé sous la juridiction de la préfecture de Tacheng.
Page enregistrée : Xian de Shawan_fr.txt
Contenu de l'article 3 (de):
Tinkoff war ein Radsportteam, das von 2001 bis 2014 mit einer dänischen und danach bis zu seiner Auflösung zum Ablauf der Saison 2016 mit russischer Lizenz betrieben wurde.
Page enregistrée : Tinkoff_de.txt
Contenu de l'article 4 (it):
James Anthony Walsh è stato un vescovo cattolico statunitense, cofondatore e primo superiore generale della Società per le missioni estere degli Stati Uniti d'America.
Page enregistrée : James Anthony Walsh_it.txt
Contenu de l'article 5 (pt):
O município de Jackson é um município localizado no condado de Muskingum no estado estadounidense de Ohio. No ano de 